In [ ]:
# 1. 아래페이지에서 순위가 1위부터 5위까지의 영화제목과 링크를 수집
# https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=cnt&date=20210520

import requests
import bs4

url = "https://movie.naver.com/movie/sdb/rank/rmovie.nhn"
response = requests.get(url)

# xml일 경우 lxml-xml ,html일 경우 html.parser
soup = bs4.BeautifulSoup(response.text,'html.parser')

arr = []
arr1 = []
for tmp in soup.select('.list_ranking > tbody > tr > td > .tit3',limit = 5):
    print('[url] ' + 'https://movie.naver.com' + tmp.select_one('a').attrs['href'],"\n"
          '[제목] ' + tmp.find('a').text)
    print('-' * 65)
    arr.append('https://movie.naver.com' + tmp.select_one('a').attrs['href'])
    arr1.append({'url':'http://movie.naver.com/' + tmp.select_one('a').attrs['href'],
               'title':tmp.find('a').text})
print(arr1)
print(arr)


In [ ]:
# 2.수집된 링크를 이용하여 상세페이지 수집(영화제목, 관람등급, 관객평점, 장르, 상영시간, 개봉일, 감독, 출연배우)
# 이 부분은 위에서 수집한 링크 주소 예시1) https://movie.naver.com/movie/bi/mi/basic.nhn?code=189150

import requests
import bs4


arr2 = []
for tmp in arr:
    url = tmp
    response = requests.get(url)
    
    soup = bs4.BeautifulSoup(response.text,'html.parser')
    
    
    title = soup.select_one("#content > div.article > div.mv_info_area > div.mv_info > h3 > a:nth-child(1)")
    print(title.text)
    age = soup.select_one('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(8) > p > a')
    print('[관람등급] ' + age.text)
    rate = soup.select_one('#actualPointPersentBasic > div > span > span')
    print('[관객평점] ' + rate.text)
    genre = soup.select_one('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(2) > p > span:nth-child(1) > a')
    print('[장르] ' + genre.text)
    time = soup.select_one('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(2) > p > span:nth-child(3)')
    print('[상영시간] ' + time.text)
    year = soup.select_one('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(2) > p > span:nth-child(4) > a:nth-child(1)')
    day = soup.select_one('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(2) > p > span:nth-child(4) > a:nth-child(2)')
    print('[개봉일] ' + year.text + day.text)
    director = soup.select_one('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(4) > p > a')
    print('[감독] ' + director.text)
    actors = soup.select_one('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(6) > p')
    print('[배우] ' + actors.text)
    print('-' * 60)
    arr2.append({'title':title.text,'age':age.text,'rate':rate.text,
                'genre':genre.text,'time':time.text,'dday':year.text+
                day.text,'director':director.text,'actors':actors.text})
print(arr2)

In [ ]:
# 3.영화의 포스트 이미지 수집

In [ ]:
# 4.수집된 자료를 몽고DB에 저장
# 컬렉션은 exam_movie (영화제목, 관람등급, 관객평점, 장르, 상영시간, 개봉일, 감독, 출연배우)
import pymongo

try:
    username = "id117"
    password = "pw117"
    host = "1.234.5.158"
    port = 37017
    dbname = "id117"


    # mongodb 접속
    # conn = pymongo.MongoClient(f"mongodb://{host}:{port}") 
    conn= pymongo.MongoClient(f'mongodb://{username}:{password}@{host}:{port}/{dbname}')

    db = conn.get_database(dbname) #db선택
    collection = db.get_collection("exam_movie") #collection == table과 비슷함

    # dict type
#     dic1 = {"id":"pyid", "pw":"aaa","name":'abc', "age":33} #딕셔너리 생성

    # list type
#     data1 = [{"id":"aaa", "name":"가다라", "age":22},
#                  {"id":"aaa", "name":"다라마", "age":32},
#                  {"id":"aaa", "name":"다라마", "age":32}]

    # mongodb에 추가
    collection.insert_one(dic1) # dic1 추가
    collection.insert_many(arr2) # data1 추가

    print('정상수행')
except Exception as e:
    print(e)
    print('오류발생시 수행됨')
finally:
    conn.close() 
    print('오류와 상관없이 수행됨')


In [33]:
# 5.수집된 자료를 HADDOP에 저장
import hdfs
import json

# 1. hdfs web으로 접속 ( 9870, 9864, 9000, 9866 )
client_hdfs = hdfs.InsecureClient('http://1.234.5.158:9870', user='hdfs')

# 2. 수집한 자료 (리스트 (딕셔너리) )

print(arr2)
# 3. 삭제하기
# client_hdfs.delete('/id100/ex_20210525.json')

# 4. 저장하기
# with는 자원을 획득하고 사용 후 반납해야 하는 경우에 사용
with client_hdfs.write('/id117/exam_movie.json', encoding = "utf-8" ) as writer:
    json.dump(arr2, writer)

# 5. 확인 크롬에서  http://1.234.5.158:9870/explorer.html#/

[{'title': '분노의 질주: 더 얼티메이트', 'age': '12세 관람가', 'rate': '관람객 평점 9.05점', 'genre': '액션', 'time': '142분 ', 'dday': ' 2021.05.19', 'director': '저스틴 린', 'actors': '빈 디젤(도미닉 토레토), 존 시나(제이콥 토레토), 성 강(한)'}, {'title': '스파이럴', 'age': '청소년 관람불가', 'rate': '관람객 평점 7.74점', 'genre': '스릴러', 'time': '93분 ', 'dday': ' 2021.05.12', 'director': '대런 린 보우즈만', 'actors': '크리스 록, 사무엘 L. 잭슨, 마리솔 니콜스'}, {'title': '극장판 귀멸의 칼날: 무한열차편', 'age': '15세 관람가', 'rate': '관람객 평점 9.62점', 'genre': '애니메이션', 'time': '117분 ', 'dday': ' 2021.01.27', 'director': '소토자키 하루오', 'actors': '하나에 나츠키(카마도 탄지로 목소리), 키토 아카리(카마도 네즈코 목소리)'}, {'title': '도라에몽: 스탠바이미 2', 'age': '전체 관람가', 'rate': '관람객 평점 9.08점', 'genre': '애니메이션', 'time': '96분 ', 'dday': ' 2021.05.19', 'director': '야기 류이치', 'actors': '윤아영(도라에몽 한국어 목소리), 김정아(진구 한국어 목소리), 김혜성(성인 진구 한국어 목소리)'}, {'title': '더 스파이', 'age': '15세 관람가', 'rate': '관람객 평점 8.78점', 'genre': '스릴러', 'time': '112분 ', 'dday': ' 2021.04.28', 'director': '도미닉 쿡', 'actors': '베네딕트 컴버배치(그레빌 윈)'}]
